In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

import torchbnn as bnn
import pymongo
import pymongo.database
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.model_selection
import sys

sys.path.append('../')
import src.IQ as IQ

## Connecting to the database

In [2]:
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/admin")
BLE = myclient["BLE"]

def query(collection, filter:dict, addFrameColumn=False):
    df =  pd.DataFrame(list(collection.find(filter)))
    if addFrameColumn:
        df['frame'] = df.apply(lambda x: x['I'] + np.dot(x['Q'],1j), axis=1)
    return df.copy()


In [3]:
def df_modify(df):
    if 'txPower' in df.columns:
        df_temp = df[['I','Q','frame','dvc','pos', 'antenna', 'txPower','SDR','frameChnl']].copy()
        df_temp['txPower'] = df.apply(lambda x: 9 if x['txPower'] == '9dbm' else 3, axis=1)
        df_temp[['dvc','pos', 'antenna', 'txPower','SDR','frameChnl']] = df_temp[['dvc','pos', 'antenna', 'txPower','SDR','frameChnl']].astype('int')
    else:
        df_temp = df[['I','Q','frame','dvc','pos', 'antenna', 'antenna_side','SDR','frameChnl']].copy()
        df_temp['antenna_side'] = df.apply(lambda x: 0 if x['antenna_side'] == 'left' else 1, axis=1)
        df_temp['pos'] = df.apply(lambda x: 0 if x['pos'] == 'static' else 1, axis=1)
        df_temp[['dvc','pos', 'antenna','SDR','frameChnl']] = df_temp[['dvc','pos', 'antenna','SDR','frameChnl']].astype('int')
    return df_temp.copy()

In [4]:
def df_to_tensor(df: pd.DataFrame):
    y = df.pop('dvc').to_numpy()-1
    X = df.to_numpy() 
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y ,
                                                                                random_state=104, 
                                                                                test_size=0.25, 
                                                                                shuffle=True
                                                                                )
    return torch.from_numpy(X_train).float(),torch.from_numpy(X_test).float(), torch.from_numpy(y_train).int(), torch.from_numpy(y_test).int()

In [6]:
df = query(BLE['offBody'], {'antenna': '1','SDR':'1','pos':'1' }, addFrameColumn=True)
df_offBody = df_modify(df)

In [7]:
# X_train, X_test, y_train, y_test = df_to_tensor(df_offBody)
# pd.DataFrame(df_offBody['I'].tolist())

y = df_offBody.pop('dvc')
X = df_offBody


In [14]:
iq = IQ.IQ(Fc=2439810000,Fs=100e6)

from itertools import groupby
from operator import itemgetter

def bitFinder(samples,farmeBiggerThan = 82, frameSmallerThan = 1000):
    test_list = np.nonzero(samples)
    framesIndex = []
    for k, g in groupby(enumerate(test_list[0]), lambda ix: ix[0]-ix[1]):
        temp = list(map(itemgetter(1), g))
        if len(temp)< farmeBiggerThan or len(temp)> frameSmallerThan:
            continue
        framesIndex.append([temp[0],temp[-1]])
    return np.array(framesIndex)

def keepPositive(samples):
    samples[samples<0] = 0
    return samples
def keepNegative(samples):
    samples[samples>0] = 0
    return samples



X['negative'] = iq.apply([bitFinder, keepNegative,iq.gradient,iq.unwrapPhase, iq.phase,iq.filter, iq.demodulate],X)
X['positive'] = iq.apply([bitFinder, keepPositive,iq.gradient,iq.unwrapPhase, iq.phase,iq.filter, iq.demodulate],X)
# sam = [1,2,3,4,0,0,0,0,0,0,0,0,3,2,3,3,20,0,2,2,2,2,0,0,0,0,0,0,2,22,22]
# frameFinder(np.array(sam))

In [15]:
X.head()

,I,Q,frame,pos,antenna,txPower,SDR,frameChnl,negative,positive
0,"[0.012604144401848316, 0.00039674062281847, -0...","[-0.0030518509447574615, 0.010620441287755966,...",[(0.012604144401848316-0.0030518509447574615j)...,1,1,9,1,33,"[[609, 735], [1235, 1336], [1433, 1535], [1637...","[[121, 313], [1089, 1234], [1337, 1432], [1536..."
1,"[0.004181035794317722, -0.010559404268860817, ...","[-0.011322367005050182, -0.001342814415693283,...","[(0.004181035794317722-0.011322367005050182j),...",1,1,9,1,4,"[[1224, 1327], [1427, 1524], [1627, 1725], [18...","[[106, 298], [1081, 1223], [1328, 1421], [1525..."
2,"[-0.005890072323381901, -0.01422162540256977, ...","[-0.00991851557046175, -0.0033570360392332077,...","[(-0.005890072323381901-0.00991851557046175j),...",1,1,9,1,12,"[[1217, 1315], [1415, 1513], [1616, 1712], [18...","[[94, 322], [1073, 1216], [1316, 1414], [1516,..."
3,"[-0.008209479041397572, -0.014404736459255219,...","[0.00674459058791399, 0.0039674062281847, -0.0...","[(-0.008209479041397572+0.00674459058791399j),...",1,1,9,1,20,"[[1246, 1345], [1443, 1546], [1646, 1744], [18...","[[118, 352], [1092, 1242], [1346, 1442], [1547..."
4,"[-0.01275673694908619, -0.005615405738353729, ...","[0.0021668141707777977, -0.011352885514497757,...",[(-0.01275673694908619+0.0021668141707777977j)...,1,1,9,1,28,"[[1226, 1331], [1426, 1526], [1626, 1726], [18...","[[109, 293], [1083, 1225], [1332, 1425], [1529..."


In [6]:


# %%
model = nn.Sequential(
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=6, out_features=1000),
    nn.ReLU(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=1000, out_features=10000),
    nn.ReLU(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=10000, out_features=1000),
    nn.ReLU(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=1000, out_features=13),
)



In [ ]:
ce_loss = nn.CrossEntropyLoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
kl_weight = 0.01

optimizer = optim.Adam(model.parameters(), lr=0.01)

# %%
kl_weight = 0.1

# %%
for step in range(3000):
    pre = model(x)
    ce = ce_loss(pre, y)
    kl = kl_loss(model)
    cost = ce + kl_weight*kl
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
_, predicted = torch.max(pre.data, 1)
total = y.size(0)
correct = (predicted == y).sum()
print('- Accuracy: %f %%' % (100 * float(correct) / total))
print('- CE : %2.2f, KL : %2.2f' % (ce.item(), kl.item()))